Day 14 of Python Summer Party

by Interview Master

Starbucks

Loyalty Program's Impact on Transaction Patterns

You are a Business Analyst on the Starbucks Rewards team investigating customer transaction behavior. Your team wants to understand how loyalty program membership influences purchasing patterns. The goal is to compare transaction metrics between loyalty members and non-members.

Question 1 of 3

For the month of July 2024, how many transactions did loyalty program members and non-members make? Compare the transaction counts between these two groups.

Question 2:
What is the average transaction value for loyalty program members and non-members during July 2024? Use this to identify which group has a higher average transaction value.

Question 3:
Determine the percentage difference in average transaction value between loyalty program members and non-members for July 2024.